# 📚 **Import Libraries**

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import tensorflow_datasets as tfds
from matplotlib.pyplot import figure

# **Using tensorflow datasets as my inputs instead of the input csv files (just for fun)**

Define my training circuits

In [ ]:
(training_images, training_labels) = tfds.as_numpy(tfds.load(
    'mnist',
    split='train[30%:]',
    batch_size=-1,
    as_supervised=True,
    shuffle_files=True,
))

training_images = training_images / 255.0

# Looking at shape of my training data.
print(training_images.shape)

# **Take a look at a set of training_images**

In [ ]:
figure(figsize=(30, 10), dpi=60)

x, y = 10, 5
for i in range(50):  
    plt.subplot(y, x, i+1)
    plt.imshow(training_images[i].reshape((28,28)),aspect='auto',interpolation='none', cmap='binary')
    plt.axis('off')
    plt.tight_layout(pad=0.0)
plt.show()

# **Closer look at a training_images image**

In [ ]:
img = training_images[0].reshape(1,28,28,1)
fig = plt.figure(figsize=(8,8))
plt.imshow(img[0,:,:,0], interpolation='none', cmap='binary')
plt.axis('off')

Define my test circuits

In [ ]:
(test_images, test_labels) = tfds.as_numpy(tfds.load(
    'mnist',
    split='train[70%:]+test',
    batch_size=-1,
    as_supervised=True,
    shuffle_files=True,
))

test_images = test_images/255.0

# Looking at shape of my testing data.
print(test_images.shape)

# **Take a look at a set of test_images**

In [ ]:
figure(figsize=(30, 10), dpi=60)

x, y = 10, 5
for i in range(50):  
    plt.subplot(y, x, i+1)
    plt.imshow(test_images[i].reshape((28,28)),aspect='auto',interpolation='none', cmap='gray')
    plt.axis('off')
    plt.tight_layout(pad=0.0)
plt.show()

# **Closer look at a training_images image**

In [ ]:
img = test_images[0].reshape(1,28,28,1)
fig = plt.figure(figsize=(8,8))
plt.imshow(img[0,:,:,0], interpolation='none', cmap='gray')
plt.axis('off')

# ***Defining the model.***

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20, verbose=1, restore_best_weights=True)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(20, (2,2), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(20, (2,2), activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(20, (2,2), activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(20, (2,2), activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(20, (3,2), activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(20, (2,3), activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(20, (3,3), activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(20, (5,2), activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(20, (2,5), activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(20, (5,5), activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(20, (3,5), activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Conv2D(20, (5,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(200, activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10, activation='softmax')
])

# **ALL THE CONVOLUTIONS!!**

In [ ]:
model.summary()

# **Listing out the model layer_names to look at the convolutions**

In [ ]:
layer_names = [layer.name for layer in model.layers]
layer_names

# **Take a look at any layer in the model by changing layer_name below.**

In [ ]:
#layer_name = 'conv2d_3'
#layer_dict = {layer.name : layer for layer in model.layers}
#modelslice = tf.keras.Model(model.inputs, layer_dict[layer_name].output)
#image = training_images[20] 
#image = np.expand_dims(image, axis=0)
#feature_maps = modelslice.predict(image)
#plt.figure(figsize=(20, 10))

#for i in range(16): # This is itterating through 16 filters of the convolution!
    #plt.subplot(4,8,i+1)
    #plt.axis('off')
    #plt.imshow(feature_maps[0, :, :, i-1], cmap='inferno')
    #plt.tight_layout(pad=0.0)

# **Compile, Fit, and Evaluate**



The callback will stop the training when there is no improvement in the
loss for 10 consecutive epochs.

The restore_best_weights=True will take the model back to its best fit. 

In [ ]:
optimizer = 'adamax'

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(training_images, 
                    training_labels,
                    epochs=500,
                    verbose=1,
                    callbacks=[callback])

test_loss, test_acc = model.evaluate(test_images, test_labels)

# **Plotting my accuracy and loss**

In [ ]:
fig = plt.figure(figsize=(20,12))

plt.subplot(211)
plt.style.use('dark_background')
plt.plot(history.history['accuracy'], color='c', label="Training accuracy")
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(loc='best')

plt.subplot(212)
plt.style.use('dark_background')
plt.plot(history.history['loss'], color='m', label="Training loss")
plt.ylabel('Loss')
plt.legend(loc='best')

# **Prepare and send the submission to the output directory**

In [ ]:
test_data = pd.read_csv('../input/digit-recognizer/test.csv')
test_data = np.array(test_data, dtype=np.float32)/255
test_data = test_data.reshape(-1,28,28,1)
prediction = model.predict(test_data)
predict = np.array(np.round(prediction), dtype = np.int32)
predict = np.argmax(predict , axis=1).reshape(-1, 1)
out = [{'ImageId': i+1, 'Label': predict[i][0]} for i in range(len(predict))]
pd.DataFrame(out).to_csv('submission.csv', index=False)

# **This is to remove a submission from the output directory**

In [ ]:
#os.remove("/kaggle/working/submission.csv")